### Reference ###

1. [Github Repo - Jinjasql](https://github.com/sripathikrishnan/jinjasql)
   
2. [simple-approach](https://towardsdatascience.com/a-simple-approach-to-templated-sql-queries-in-python-adc4f0dc511)
   
3. [advanced-sql-templates](https://towardsdatascience.com/advanced-sql-templates-in-python-with-jinjasql-b996eadd761d)

In [ ]:
!pip3 install jinjasql

In [2]:
user_transaction_template = '''
select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = {{ user_id }}
    and transaction_date = {{ transaction_date }}
group by
    user_id
'''

In [3]:
params = {
    'user_id': 1234,
    'transaction_date': '2019-03-02',
}

In [4]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')
query, bind_params = j.prepare_query(user_transaction_template, params)

In [5]:
print(query)


select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = %(user_id_1)s
    and transaction_date = %(transaction_date_2)s
group by
    user_id


In [6]:
print(bind_params)

{'user_id_1': 1234, 'transaction_date_2': '2019-03-02'}


In [7]:
print(query % bind_params)



select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = 1234
    and transaction_date = 2019-03-02
group by
    user_id


In [14]:
from six import string_types


In [15]:
from six import string_types
def quote_sql_string(value):
    '''
    If `value` is a string type, escapes single quotes in the string
    and returns the string enclosed in single quotes.
    '''
    if isinstance(value, string_types):
        new_value = str(value)
        new_value = new_value.replace("'", "''")
        return "'{}'".format(new_value)
    return value

In [17]:
from copy import deepcopy
def get_sql_from_template(query, bind_params):
    if not bind_params:
        return query
    params = deepcopy(bind_params)
    for key, val in params.items():
        params[key] = quote_sql_string(val)
    return query % params

In [18]:
print(get_sql_from_template(query, bind_params))


select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = 1234
    and transaction_date = '2019-03-02'
group by
    user_id


In [19]:

def apply_sql_template(template, parameters):
    '''
    Apply a JinjaSql template (string) substituting parameters (dict) and return
    the final SQL.
    '''
    j = JinjaSql(param_style='pyformat')
    query, bind_params = j.prepare_query(template, parameters)
    return get_sql_from_template(query, bind_params)

In [20]:
COLUMN_STATS_TEMPLATE = '''
select
    {{ column_name | sqlsafe }} as column_name
    , count(*) as num_rows
    , count(distinct {{ column_name | sqlsafe }}) as num_unique
    , sum(case when {{ column_name | sqlsafe }} is null then 1 else 0 end) as num_nulls
    {% if default_value %}
    , sum(case when {{ column_name | sqlsafe }} = {{ default_value }} then 1 else 0 end) as num_default
    {% else %}
    , 0 as num_default
    {% endif %}
    , min({{ column_name | sqlsafe }}) as min_value
    , max({{ column_name | sqlsafe }}) as max_value
from
    {{ table_name | sqlsafe }}
'''

In [21]:
def get_column_stats_sql(table_name, column_name, default_value):
    '''
    Returns the SQL for computing column statistics.
    Passing None for the default_value results in zero output for the number
    of default values.
    '''
    params = {
        'table_name': table_name,
        'column_name': column_name,
        'default_value': default_value,
    }
    return apply_sql_template(COLUMN_STATS_TEMPLATE, params)

In [22]:
print(get_column_stats_sql('transactions', 'user_id', None))


select
    user_id as column_name
    , count(*) as num_rows
    , count(distinct user_id) as num_unique
    , sum(case when user_id is null then 1 else 0 end) as num_nulls
    
    , 0 as num_default
    
    , min(user_id) as min_value
    , max(user_id) as max_value
from
    transactions
